In [ ]:
import os
import cv2
from keras.applications import Xception
from keras.layers import Dense, Flatten
from keras.models import Sequential
import pandas as pd
from keras.utils import image_dataset_from_directory

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf

In [ ]:
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True))

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5



In [ ]:
model = Sequential()
model.add(Xception(weights='imagenet', include_top=False, input_shape=(224, 224, 3), classes=2))
model.add(Flatten())
model.add(Dense(256, activation='sigmoid'))
model.add(Dense(128, activation='sigmoid'))
model.add(Dense(64, activation='sigmoid'))
model.add(Dense(2, activation='softmax'))

83683744/83683744 [==============================] - 5s 0us/step


In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 xception (Functional)       (None, 7, 7, 2048)        20861480  
                                                                 
 flatten (Flatten)           (None, 100352)            0         
                                                                 
 dense (Dense)               (None, 256)               25690368  
                                                                 
 dense_1 (Dense)             (None, 128)               32896     
                                                                 
 dense_2 (Dense)             (None, 64)                8256      
                                                                 
 dense_3 (Dense)             (None, 2)                 130       
                                                                 
Total params: 46,593,130
Trainable params: 46,538,602
No

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy', tf.keras.metrics.CategoricalAccuracy(), tf.keras.metrics.Recall(), tf.keras.metrics.Precision()])

In [ ]:
train_ds = image_dataset_from_directory(
    directory='/content/drive/My Drive/IndoorOutdoorClassification/images',
    labels='inferred',
    label_mode='categorical',
    image_size=(224, 224),
    shuffle=True,
    seed=42,
    validation_split=0.2,
    subset='training'
)

Found 800 files belonging to 2 classes.
Using 640 files for training.


In [ ]:
val_ds = image_dataset_from_directory(
    directory='/content/drive/My Drive/IndoorOutdoorClassification/images',
    labels='inferred',
    label_mode='categorical',
    image_size=(224, 224),
    shuffle=True,
    seed=42,
    validation_split=0.2,
    subset='validation'
)

Found 800 files belonging to 2 classes.
Using 160 files for validation.


In [ ]:
epochs=100
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

In [ ]:
save_directory='/content/drive/My Drive/IndoorOutdoorClassification/xception_nodropout.h5'
model.save(save_directory)

## Evaluation

In [ ]:
import numpy as np

In [ ]:
def predict_image_softmax(model, path: str,  size_x: int = 224, size_y: int = 224):
  img = cv2.imread(path)
  img = cv2.resize(img, (size_x, size_y))
  img = np.expand_dims(img, axis=0)
  return model.predict(img, verbose=False)[0]

In [ ]:
prediction_dir = '/content/drive/My Drive/IndoorOutdoorClassification/images/indoor'
cnt_zero = 0
cnt_one = 0
imgs = os.listdir(prediction_dir)

for img in imgs:
  try:
    prediction = predict_image_softmax(model,f'{prediction_dir}/{img}')
    print(f'{img} - {prediction}')
    if(prediction[0]>=prediction[1]):
      cnt_zero += 1
    else:
      cnt_one += 1
  except Exception:
    print('except')
print(f"Class zero: {cnt_zero} \n Class one: {cnt_one}")

In [ ]:
prediction_dir = '/content/drive/My Drive/IndoorOutdoorClassification/images/outdoor'
cnt_zero = 0
cnt_one = 0
imgs = os.listdir(prediction_dir)

for img in imgs:
  try:
    prediction = predict_image_softmax(model,f'{prediction_dir}/{img}')
    print(f'{img} - {prediction}')
    if(prediction[0]>=prediction[1]):
      cnt_zero += 1
    else:
      cnt_one += 1
  except Exception:
    print('except')
print(f"Class zero: {cnt_zero} \n Class one: {cnt_one}")

In [ ]:
prediction_dir = '/content/drive/My Drive/IndoorOutdoorClassification/prediction_images/'
cnt_zero = 0
cnt_one = 0
imgs = os.listdir(prediction_dir)

for img in imgs:
  try:
    prediction = predict_image_softmax(model,f'{prediction_dir}/{img}')
    print(f'{img} - {prediction} - {"indoor" if prediction[0]>=prediction[1] else "outdoor"}')
    if(prediction[0]>=prediction[1]):
      cnt_zero += 1
    else:
      cnt_one += 1
  except Exception:
    print('except')
print(f"Class zero: {cnt_zero} \n Class one: {cnt_one}")

indoor1.jpg - [9.9964786e-01 3.5211822e-04] - indoor
indoor2.jpg - [9.9961555e-01 3.8438314e-04] - indoor
outdoor1.jpg - [2.7296698e-04 9.9972707e-01] - outdoor
outdoor2.jpg - [0.96582985 0.03417015] - indoor
outdoor3.jpg - [0.00307955 0.99692047] - outdoor
outdoor4.jpg - [0.36660567 0.6333943 ] - outdoor
indoor3.jpg - [9.9964106e-01 3.5894307e-04] - indoor
indoor4.jpg - [9.9962056e-01 3.7936808e-04] - indoor
Class zero: 5 
 Class one: 3


## With dropout

In [ ]:
from keras.layers import Dropout


In [ ]:
model1 = Sequential()
model1.add(Xception(weights='imagenet', include_top=False, input_shape=(224, 224, 3), classes=2))
model1.add(Flatten())
model1.add(Dense(128, activation='sigmoid'))
model1.add(Dropout(0.3))
model1.add(Dense(64, activation='sigmoid'))
model1.add(Dropout(0.1))
model1.add(Dense(2, activation='softmax'))

In [ ]:
model1.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 xception (Functional)       (None, 7, 7, 2048)        20861480  
                                                                 
 flatten_1 (Flatten)         (None, 100352)            0         
                                                                 
 dense_3 (Dense)             (None, 128)               12845184  
                                                                 
 dropout_2 (Dropout)         (None, 128)               0         
                                                                 
 dense_4 (Dense)             (None, 64)                8256      
                                                                 
 dropout_3 (Dropout)         (None, 64)                0         
                                                                 
 dense_5 (Dense)             (None, 2)                

In [ ]:
model1.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy', tf.keras.metrics.CategoricalAccuracy(), tf.keras.metrics.Recall(), tf.keras.metrics.Precision()])

In [ ]:
train_ds = image_dataset_from_directory(
    directory='/content/drive/My Drive/IndoorOutdoorClassification/images',
    labels='inferred',
    label_mode='categorical',
    image_size=(224, 224),
    shuffle=True,
    seed=42,
    validation_split=0.2,
    subset='training'
)

Found 800 files belonging to 2 classes.
Using 640 files for training.


In [ ]:
val_ds = image_dataset_from_directory(
    directory='/content/drive/My Drive/IndoorOutdoorClassification/images',
    labels='inferred',
    label_mode='categorical',
    image_size=(224, 224),
    shuffle=True,
    seed=42,
    validation_split=0.2,
    subset='validation'
)

Found 800 files belonging to 2 classes.
Using 160 files for validation.


In [ ]:
epochs=100
history = model1.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

Epoch 1/100
20/20 [==============================] - 206s 4s/step - loss: 0.3475 - accuracy: 0.8703 - categorical_accuracy: 0.8703 - recall: 0.8703 - precision: 0.8703 - val_loss: 0.4157 - val_accuracy: 0.8438 - val_categorical_accuracy: 0.8438 - val_recall: 0.8438 - val_precision: 0.8438
Epoch 2/100
20/20 [==============================] - 12s 582ms/step - loss: 0.1340 - accuracy: 0.9625 - categorical_accuracy: 0.9625 - recall: 0.9625 - precision: 0.9625 - val_loss: 0.6470 - val_accuracy: 0.8062 - val_categorical_accuracy: 0.8062 - val_recall: 0.8062 - val_precision: 0.8062
Epoch 3/100
20/20 [==============================] - 13s 601ms/step - loss: 0.1944 - accuracy: 0.9484 - categorical_accuracy: 0.9484 - recall: 0.9484 - precision: 0.9484 - val_loss: 1.0876 - val_accuracy: 0.6938 - val_categorical_accuracy: 0.6938 - val_recall: 0.6938 - val_precision: 0.6938
Epoch 4/100
20/20 [==============================] - 13s 613ms/step - loss: 0.1321 - accuracy: 0.9563 - categorical_accuracy: 

In [ ]:
save_directory='/content/drive/My Drive/IndoorOutdoorClassification/xception_dropout.h5'
model1.save(save_directory)

In [ ]:
import numpy as np

In [ ]:
def predict_image_softmax(model, path: str,  size_x: int = 224, size_y: int = 224):
  img = cv2.imread(path)
  img = cv2.resize(img, (size_x, size_y))
  img = np.expand_dims(img, axis=0)
  return model.predict(img, verbose=False)[0]

In [ ]:
prediction_dir = '/content/drive/My Drive/IndoorOutdoorClassification/prediction_images/'
cnt_zero = 0
cnt_one = 0
imgs = os.listdir(prediction_dir)

for img in imgs:
  # try:
    prediction = predict_image_softmax(model1,f'{prediction_dir}/{img}')
    print(f'{img} - {prediction} - {"indoor" if prediction[0]>=prediction[1] else "outdoor"}')
    if(prediction[0]>=prediction[1]):
      cnt_zero += 1
    else:
        cnt_one += 1
  # except Exception:
  #   print('except')
print(f"Class zero: {cnt_zero} \n Class one: {cnt_one}")

indoor1.jpg - [9.9920964e-01 7.9036446e-04] - indoor
indoor2.jpg - [0.99611497 0.00388503] - indoor
outdoor1.jpg - [0.00740838 0.9925916 ] - outdoor
outdoor2.jpg - [9.9922454e-01 7.7548122e-04] - indoor
outdoor3.jpg - [0.991178   0.00882195] - indoor
outdoor4.jpg - [0.9919574  0.00804254] - indoor
indoor3.jpg - [0.9986255  0.00137454] - indoor
indoor4.jpg - [9.9925345e-01 7.4654637e-04] - indoor
Class zero: 7 
 Class one: 1
